In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd '..'

/mnt/scratch/l_singh/hdb/project_dir/histonedb/CURATED_SET


In [3]:
from curated_set_services import CuratedSet, GFFwrite
from curated_set_services import show_msa_jl, dict2tree, muscle_p2p_aln

import json
import pandas as pd
from ete3 import Tree, TreeStyle

In [4]:
with open('classification.json') as json_file:
    data = json.load(json_file)

In [9]:
hist_tree=Tree()
dict2tree(hist_tree,data['tree'])
print(hist_tree.get_ascii(show_internal=True))


                                                                           /-cH2A.1_(Homo_sapiens)
                                                                          |
                                                                          |--cH2A.2_(Homo_sapiens)
                                                                          |
                                                                          |--cH2A.3_(Homo_sapiens)
                                                                          |
                                                                          |--cH2A.4_(Homo_sapiens)
                                                                          |
                                                                          |--cH2A.5_(Homo_sapiens)
                                                                          |
                                                        /cH2A_(Homo_sapiens)-cH2A.6_(Homo_sapiens)
                         

In [6]:
for node in hist_tree.traverse("levelorder"):
    if node.is_root(): continue
    print(f'{"type" if node.up.is_root() else "variant_group" if node.up.up.is_root() else "variant"}', end=' ')
    print(f'name = {node.name}, parent = {node.up.name}')

type name = H2A, parent = 
type name = H2B, parent = 
type name = H3, parent = 
type name = H4, parent = 
type name = H1, parent = 
type name = Archaeal, parent = 
type name = Viral, parent = 
variant_group name = cH2A, parent = H2A
variant_group name = H2A.Z, parent = H2A
variant_group name = H2A.X, parent = H2A
variant_group name = macroH2A, parent = H2A
variant_group name = short_H2A, parent = H2A
variant_group name = H2A.R, parent = H2A
variant_group name = H2A.J, parent = H2A
variant_group name = H2A.W, parent = H2A
variant_group name = H2A.M, parent = H2A
variant_group name = gH2A, parent = H2A
variant_group name = cH2B, parent = H2B
variant_group name = H2B.L, parent = H2B
variant_group name = H2B.W, parent = H2B
variant_group name = H2B.K, parent = H2B
variant_group name = H2B.N, parent = H2B
variant_group name = H2B.O, parent = H2B
variant_group name = H2B.S, parent = H2B
variant_group name = gH2B, parent = H2B
variant_group name = H2B.Z, parent = H2B
variant_group name = H2B.

### Create SQL-query for adding histones types and variants with description including references and alternative names

Information extracted from classification.json

In [7]:
sql_query = []
desc_id = 0
pubs_unique = []
alt_name_id = 0
added_histones = []
for node in hist_tree.traverse("levelorder"):
    if node.is_root(): continue
    added_histones.append(node.name)
    # insert desc
    desc = None if node.name not in data["info"].keys() else data["info"][node.name].get("description", None)
    if type(desc) is str:
        desc = None if desc == "null" else {'summary': desc}
    if desc is not None:
        desc_id += 1
        sql_query.append(f'''INSERT INTO `curatedhitdb`.`histone_description`
(`id`,
`summary`,
`taxonomy`,
`genes`,
`evolution`,
`expression`,
`knock_out`,
`function`,
`sequence`,
`localization`,
`deposition`,
`structure`,
`interactions`,
`disease`,
`caveats`)
VALUES
({desc_id},
'{"null" if desc is None else desc.get("summary", "null").replace("--", "").replace("'", "''")}',
'{"null" if desc is None else desc.get("taxonomy", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("genes", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("evolution", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("expression", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("knock_out", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("function", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("sequence", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("localization", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("deposition", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("structure", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("interactions", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("disease", "null").replace("'", "''")}',
'{"null" if desc is None else desc.get("caveats", "null").replace("'", "''")}');''')
    # insert histone
    sql_query.append(f'''INSERT INTO `curatedhitdb`.`histone`
(`id`,
`level`,
`taxonomic_span`,
`taxonomic_span_id`,
`description`,
`parent`)
VALUES
('{node.name}',
'{"type" if node.up.is_root() else "variant_group" if node.up.up.is_root() else "variant"}',
'{"null" if node.name not in data["info"].keys() else data["info"][node.name].get("taxonomic_span", "null")}',
'{"null" if node.name not in data["info"].keys() else data["info"][node.name].get("taxonomic_span_id", "null")}',
{desc_id if desc is not None else "null"},
{"null" if node.up.is_root() else f"'{node.up.name}'"});''')
    # insert publication (by default all feild will be null except PK)
    pubs = None if node.name not in data["info"].keys() else data["info"][node.name].get("publications", None)
    if pubs is not None:
        if type(pubs) != list: print(f"PUBS IS NOT A LIST:\n{pubs}")
        else:
            for p in pubs:
                if p not in pubs_unique:
                    pubs_unique.append(p)
                    sql_query.append(f'''INSERT INTO `curatedhitdb`.`publication`
(`id`,
`title`,
`doi`,
`author`,
`year`)
VALUES
('{p}',
null,
null,
null,
null);''')
                # insert relation histone_has_publication
                sql_query.append(f'''INSERT INTO `curatedhitdb`.`histone_has_publication`
(`histone_id`,
`publication_id`)
VALUES
('{node.name}',
'{p}');''')
    # insert alternative name
    alts = None if node.name not in data["info"].keys() else data["info"][node.name].get("alternate_names", None)
    if alts is not None:
        if type(alts) != list: print(f"ALTS IS NOT A LIST:\n{alts}")
        else:
            for an in alts:
                alt_name_id += 1
                if "name" not in an.keys(): print(f"ALT NAME DOES NOT EXIST:\n{an}")
                sql_query.append(f'''INSERT INTO `curatedhitdb`.`alternative_name`
(`id`,
`name`,
`taxonomy`,
`gene`,
`splice`,
`histone`)
VALUES
({alt_name_id},
'{an["name"]}',
'{an.get("taxonomy", "null")}',
'{an.get("gene", "null")}',
'{an.get("splice", "null")}',
'{node.name}');''')
# print('\n'.join(sql_query).replace("'null'", "null"))

In [8]:
with open('curated_service/sql_query_histones.sql', 'w') as f:
    f.write('\n'.join(sql_query).replace("'null'", "null"))

### Create SQL-query for adding histones sequences including references

Information extracted from histones.csv

This code uses `pubs_unique` from previous step because these references are already included to publication table at previous step.

This code uses `added_histones` from previous step because some variant names from csv-table does not exist in database histone table.

In [10]:
df_seq = pd.read_csv('histones.csv')
df_seq.index = df_seq.accession
df_seq

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
accession,,,,,,,,,,,,,,,,
AAA30025.1,AAA30025.1,H2B,cH2B,cH2B_(Animals),NaN,161386,NaN,NaN,7660.0,Psammechinus miliaris,Echinodermata,Echinoidea,NaN,NaN,NaN,MAPTAQVAKKGSKKAVKAPRPSGGKKRNRKRKESYGIYIYKVLKQV...
AAB48832.1,AAB48832.1,H2B,CS_H2B_(Echinoidea),CS_H2B_(Echinoidea),NaN,NaN,NaN,NaN,7660.0,Psammechinus miliaris,Echinodermata,Echinoidea,NaN,NaN,9032246,MPAKGAATKGEKKQAVKSKAMASSRTGDKKRRRRRLESYNIYIYKV...
AAC28846.1,AAC28846.1,H2A,macroH2A,macroH2A,NaN,3395780,NaN,NaN,9031.0,Gallus gallus,Chordata,Aves,NaN,NaN,22650316 26989147,MSSRGGKKKSTKTSRSAKAGVIFPVGRMLRYIKKGHPKYRIGVGAP...
AAC37291.1,AAC37291.1,H2A,H2A.X,H2A.X_(Protists),NaN,310870,NaN,NaN,5911.0,Tetrahymena thermophila,Ciliophora,Oligohymenophorea,NaN,NaN,22650316 26989147,MSTTGKGGKAKGKTASSKQVSRSARAGLQFPVGRISRFLKHGRYSE...
AAC37292.1,AAC37292.1,H2A,cH2A,cH2A_(Protists),NaN,310872,NaN,NaN,5911.0,Tetrahymena thermophila,Ciliophora,Oligohymenophorea,NaN,NaN,22650316 26989147,MSTTGKGGKAKGKTASSKQVSRSARAGLQFPVGRISRFLKNGRYSE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XP_846259.1,XP_846259.1,H2A,H2A.Z,H2A.Z,NaN,72391930,NaN,NaN,185431.0,Trypanosoma brucei brucei TREU927,Euglenozoa,Kinetoplastea,NaN,NaN,22650316 26989147,MSLTGDDAVPQAPLVGGVAMSPEQASALTGGKLGGKAVGPAHGKGK...
XP_875023.2,XP_875023.2,H2A,short_H2A,H2A.L,NaN,119923405,NaN,NaN,9913.0,Bos taurus,Chordata,Mammalia,NaN,NaN,22650316 26989147,MSPRRHLWNCRRSRRHSLSRSTRAELQFPVSRVDRLLREGQGAYRL...
XP_951561.1,XP_951561.1,H4,cH4,cH4,NaN,84043542,NaN,NaN,185431.0,Trypanosoma brucei brucei TREU927,Euglenozoa,Kinetoplastea,NaN,NaN,NaN,MAKGKRVGESKGAQKRQKKVLRDNVRGITRGSIRRLARRAGVKRIS...


In [11]:
df_seq.dtypes

accession          object
type               object
variant_group      object
variant            object
doublet           float64
gi                 object
ncbi_gene_id      float64
hgnc_gene_name     object
taxonomy_id       float64
organism           object
phylum             object
class              object
taxonomy_group     object
info               object
references         object
sequence           object
dtype: object

In [12]:
df_seq.variant.isna().sum()

0

In [13]:
df_seq.sequence.isna().sum()

0

In [14]:
df_seq.references.isna().sum()

175

In [15]:
max(df_seq.sequence.apply(len))

448

In [16]:
pubs_unique

['luger_crystal_1997',
 'malik_phylogenomics_2003',
 'jiang_evolution_2020',
 'marzluff_metabolism_2008',
 'talbert_histone_2021',
 'marzluff_birth_2017',
 'pontarotti_long-term_2009',
 'giaimo_histone_2019',
 'colino-sanguino_h2az-nuclesome_2021',
 '22650316',
 '25731851',
 '11101893',
 '18275809',
 '24311584',
 '19193230',
 '24969791',
 '22467210',
 '20003410',
 '24768041',
 '23301656',
 '20197778',
 'dalmasso_canonical_2011',
 'shaytan_nucleosome_2015',
 'adam_h2az_2001',
 '18095327',
 'millar_organizing_2013',
 ' talbert_histone_2021',
 'talbert_histone_2010',
 '16107708',
 '20543561',
 '16803903',
 'sun_histone_2019',
 'molaro_evolutionary_2018',
 'chew_short_2021',
 'jiang_short_2020',
 'nishida_novel_2005',
 'contrepois_histone_2017',
 'isermann_histone_2020',
 'kawashima_diversification_2015',
 'alvarez-venegas_canonical_2019',
 'ueda_male_2005',
 '11892742',
 '22156475',
 'aul_major_2001',
 'tran_involvement_2012',
 'govin_pericentric_2007',
 'raman_novel_2022',
 'churikov_nov

In [17]:
added_histones

['H2A',
 'H2B',
 'H3',
 'H4',
 'H1',
 'Archaeal',
 'Viral',
 'cH2A',
 'H2A.Z',
 'H2A.X',
 'macroH2A',
 'short_H2A',
 'H2A.R',
 'H2A.J',
 'H2A.W',
 'H2A.M',
 'gH2A',
 'cH2B',
 'H2B.L',
 'H2B.W',
 'H2B.K',
 'H2B.N',
 'H2B.O',
 'H2B.S',
 'gH2B',
 'H2B.Z',
 'H2B.V',
 'CS_H2B_(Echinoidea)',
 'early_H2B_(Echinoidea)',
 'late_H2B_(Echinoidea)',
 'sperm_H2B_(Echinoidea)',
 'cH3',
 'H3.3',
 'H3.4_(Mammalia)',
 'H3.3-like?',
 'H3.1-like_(Plants)',
 'H3.5_(Primates_or_Hominids?)',
 'H3.6_(Mammals?)?',
 'H3.7_(Mammals?)?',
 'H3.8_(Mammals?)?',
 'H3.P_(Moneuplotes?)',
 'H3.V_(Trypanosomes?)',
 'H3.B_(Giardia?)',
 'cenH3_(Eukarya)',
 'cH4',
 'cH1',
 'generic_H1',
 'H1.0',
 'H1.1',
 'H1.2',
 'H1.3',
 'H1.4',
 'H1.5',
 'TS_H1.6',
 'TS_H1.7',
 'OO_H1.8',
 'TS_H1.9',
 'H1.10',
 'scH1',
 'cH2A_(Animals)',
 'cH2A_(Plants)',
 'cH2A_(Fungi)',
 'cH2A_(Protists)',
 'H2A.Z.1_(Chordata)',
 'H2A.Z.2_(Chordata)',
 'H2A.X_(Animals)',
 'H2A.X_(Plants)',
 'H2A.X_(Fungi)',
 'H2A.X_(Protists)',
 'macroH2A.1_(Mammalia)

In [18]:
pd.isna(df_seq.loc['NP_005309.1', 'gi'])

True

In [19]:
sql_query_seq = []
for i, row in df_seq.iterrows():
    # insert sequence
    sql_query_seq.append(f'''INSERT INTO `curatedhitdb`.`sequence`
(`accession`,
`variant`,
`gi`,
`ncbi_gene_id`,
`hgnc_gene_name`,
`taxonomy_id`,
`organism`,
`phylum`,
`class`,
`taxonomy_group`,
`info`,
`sequence`,
`variant_under_consideration`)
VALUES
('{row["accession"]}',
'{"null" if row["variant"] not in added_histones else row["variant"]}',
'{"null" if pd.isna(row["gi"]) else row["gi"]}',
'{"null" if pd.isna(row["ncbi_gene_id"]) else int(row["ncbi_gene_id"])}',
'{"null" if pd.isna(row["hgnc_gene_name"]) else row["hgnc_gene_name"]}',
{"null" if pd.isna(row["taxonomy_id"]) else int(row["taxonomy_id"])},
'{"null" if pd.isna(row["organism"]) else row["organism"]}',
'{"null" if pd.isna(row["phylum"]) else row["phylum"]}',
'{"null" if pd.isna(row["class"]) else row["class"]}',
'{"null" if pd.isna(row["taxonomy_group"]) else row["taxonomy_group"]}',
'{"null" if pd.isna(row["info"]) else row["info"]}',
'{row["sequence"]}',
'{"null" if row["variant"] in added_histones else row["variant"]}');''')
    # insert publication if not exist (by default all feild will be null except PK)
    pubs_seq = row["references"]
    if not pd.isna(pubs_seq) and pubs_seq != '' and pubs_seq != '?':
        for p in pubs_seq.split():
            p = p.replace('(?)', '')
            if p not in pubs_unique:
                pubs_unique.append(p)
                sql_query_seq.append(f'''INSERT INTO `curatedhitdb`.`publication`
(`id`,
`title`,
`doi`,
`author`,
`year`)
VALUES
('{p}',
null,
null,
null,
null);''')
            # insert relation sequence_has_publication
            sql_query_seq.append(f'''INSERT INTO `curatedhitdb`.`sequence_has_publication`
(`sequence_accession`,
`publication_id`)
VALUES
('{row["accession"]}',
'{p}');''')
print('\n'.join(sql_query_seq).replace("'null'", "null"))

INSERT INTO `curatedhitdb`.`sequence`
(`accession`,
`variant`,
`gi`,
`ncbi_gene_id`,
`hgnc_gene_name`,
`taxonomy_id`,
`organism`,
`phylum`,
`class`,
`taxonomy_group`,
`info`,
`sequence`,
`variant_under_consideration`)
VALUES
('AAA30025.1',
'cH2B_(Animals)',
'161386',
null,
null,
7660,
'Psammechinus miliaris',
'Echinodermata',
'Echinoidea',
null,
null,
'MAPTAQVAKKGSKKAVKAPRPSGGKKRNRKRKESYGIYIYKVLKQVHPDTGISSRAMIIMNSFVNDIFERIAGESSRLAQYNKKSTISSREIQTAVRLILPGELAKHAVSEGTKAVTKYTTSK',
null);
INSERT INTO `curatedhitdb`.`sequence`
(`accession`,
`variant`,
`gi`,
`ncbi_gene_id`,
`hgnc_gene_name`,
`taxonomy_id`,
`organism`,
`phylum`,
`class`,
`taxonomy_group`,
`info`,
`sequence`,
`variant_under_consideration`)
VALUES
('AAB48832.1',
'CS_H2B_(Echinoidea)',
null,
null,
null,
7660,
'Psammechinus miliaris',
'Echinodermata',
'Echinoidea',
null,
null,
'MPAKGAATKGEKKQAVKSKAMASSRTGDKKRRRRRLESYNIYIYKVLKQVHPDTGISSKAMSIMNSFVNDIFERIAAEASRLAQYNKKSTISSREVQTAVRLLLPGELAKHAVSEGTKAVTKYTTSR',
null);
INSERT INTO `curate

In [20]:
with open('curated_service/sql_query_sequences.sql', 'w') as f:
    f.write('\n'.join(sql_query_seq).replace("'null'", "null"))

In [ ]:
INSERT INTO `curatedhitdb`.`sequence_has_publication` (`sequence_accession`, `publication_id`) VALUES ('NP_505463.1', '26989147')
